In [2]:
import pandas as pd
import numpy as np

In [3]:
# Load file
schools_complete = "schools_complete.csv"
students_complete = "students_complete.csv"

In [4]:
# Read School and Student Data File and store into Pandas DataFrames
schools_data = pd.read_csv(schools_complete)
students_data = pd.read_csv(students_complete)

In [5]:
# Combine the data into a single dataset.  
schools_complete = pd.merge(students_data, schools_data, how="left", on=["school_name", "school_name"])

In [6]:
schools_complete.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


In [7]:
#Total schools
total_schools = schools_complete["school_name"].nunique()
total_schools

15

In [8]:
#Total students
total_students = schools_complete["student_name"].count()
total_students

39170

In [9]:
#Total budget
total_budget = pd.unique(schools_complete['budget'].values)
budget_final = total_budget.sum()
budget_final

24649428

In [10]:
#Average math score
avr_math_score = schools_complete['math_score'].mean()
avr_math = round (avr_math_score , 2)
avr_math

78.99

In [11]:
#Average reading score
avr_reading_score = schools_complete['reading_score'].mean()
avr_reading = round (avr_reading_score , 2)
avr_reading

81.88

In [12]:
#% passing math (the percentage of students who passed math)
pass_maths = schools_complete['math_score']>=70
final_maths = schools_complete[pass_maths]
final_maths['math_score'].count()
final_no_maths = final_maths['math_score'].count()
per_pass_maths = final_no_maths / total_students * 100 
round_maths = round (per_pass_maths , 2)
round_maths

74.98

In [13]:
#% passing reading (the percentage of students who passed reading)
pass_reading = schools_complete['reading_score']>=70
final_reading = schools_complete[pass_reading]
final_reading['reading_score'].count()
final_no_reading = final_reading['reading_score'].count()
per_pass_reading = final_no_reading / total_students * 100
round_reading = round (per_pass_reading , 2)
round_reading

85.81

In [14]:
#% overall passing (the percentage of students who passed math AND reading)
pass_overall = schools_complete[(schools_complete['math_score']>=70) & (schools_complete['reading_score']>=70)] 
total_overall = pass_overall['reading_score'].count()
per_pass_overall = total_overall / total_students * 100
round_overall = round(per_pass_overall, 2)
round_overall

65.17

In [15]:
#High-level snapshot
results_list = [[total_schools,total_students,budget_final,avr_math,avr_reading,round_maths,round_reading,round_overall]]

df = pd.DataFrame(results_list, index=['District Summary'], columns=['Total number of schools','Total number of students','Total budget','Average math score','Average reading score','Passing math score (%)','Passing reading score (%)', 'Passing math & reading score (%)'])
df.head()

,Total number of schools,Total number of students,Total budget,Average math score,Average reading score,Passing math score (%),Passing reading score (%),Passing math & reading score (%)
District Summary,15,39170,24649428,78.99,81.88,74.98,85.81,65.17


In [16]:
# School name
school = schools_complete.drop_duplicates(subset=['school_name'])
total_students_school = schools_complete["school_name"].value_counts()

In [17]:
# School type
school_summary = school[["school_name","type",]]

In [18]:
# Total students & budget
school_df = schools_complete.groupby(['school_name','type','budget'], sort=False).size().reset_index(name='Total students')

In [19]:
# Per student budget
school_df['Per student budget'] = school_df['budget'] / school_df['Total students']

In [20]:
# Average math score
avg_math_df = schools_complete.groupby('school_name', sort=False)['math_score'].mean().reset_index(name='Average math score')
avg_math = avg_math_df['Average math score']
school_df.insert(5, 'Average math score per school', avg_math)

In [21]:
# Average reading score
avg_reading_df = schools_complete.groupby('school_name', sort=False)['reading_score'].mean().reset_index(name='Average reading score')
avg_reading = avg_reading_df['Average reading score']
school_df.insert(6, 'Average reading score per school', avg_reading)

In [22]:
# % passing math (the percentage of students who passed math)
math_pass_df = final_maths.groupby('school_name', sort=False)['math_score'].count().reset_index(name='Math students passed')
math_pass = math_pass_df['Math students passed']
school_df.insert(7, 'Math students passed', math_pass)

school_df['Passing math per school (%)'] = round(school_df['Math students passed'] / school_df['Total students'] * 100, 2)
school_df = school_df.drop(columns=['Math students passed'])

In [23]:
# % passing reading (the percentage of students who passed reading
reading_pass_df = final_reading.groupby('school_name', sort=False)['reading_score'].count().reset_index(name='Reading students passed')
reading_pass = reading_pass_df['Reading students passed']
school_df.insert(8, 'Reading students passed', reading_pass)

school_df['Passing reading per school (%)'] = round(school_df['Reading students passed'] / school_df['Total students'] * 100, 2)
school_df = school_df.drop(columns=['Reading students passed'])

In [24]:
# % overall passing (the percentage of students who passed math AND reading)
total_pass_df = pass_overall.groupby('school_name', sort=False)['reading_score'].count().reset_index(name='Combined students passed')
total_pass = total_pass_df['Combined students passed']
school_df.insert(9, 'Combined students passed', total_pass)

school_df['Passing math and reading score (%)'] = round(school_df['Combined students passed'] / school_df['Total students'] * 100, 2)
school_df = school_df.drop(columns=['Combined students passed'])

In [25]:
# Final cosmetic changes
school_df = school_df.rename(columns={"school_name":"School name","type":"School type", "budget":"School budget"})
school_df

,School name,School type,School budget,Total students,Per student budget,Average math score per school,Average reading score per school,Passing math per school (%),Passing reading per school (%),Passing math and reading score (%)
0,Huang High School,District,1910635,2917,655.0,76.629414,81.182722,65.68,81.32,53.51
1,Figueroa High School,District,1884411,2949,639.0,76.711767,81.158020,65.99,80.74,53.20
2,Shelton High School,Charter,1056600,1761,600.0,83.359455,83.725724,93.87,95.85,89.89
3,Hernandez High School,District,3022020,4635,652.0,77.289752,80.934412,66.75,80.86,53.53
4,Griffin High School,Charter,917500,1468,625.0,83.351499,83.816757,93.39,97.14,90.60
5,Wilson High School,Charter,1319574,2283,578.0,83.274201,83.989488,93.87,96.54,90.58
6,Cabrera High School,Charter,1081356,1858,582.0,83.061895,83.975780,94.13,97.04,91.33
7,Bailey High School,District,3124928,4976,628.0,77.048432,81.033963,66.68,81.93,54.64
8,Holden High School,Charter,248087,427,581.0,83.803279,83.814988,92.51,96.25,89.23
9,Pena High School,Charter,585858,962,609.0,83.839917,84.044699,94.59,95.95,90.54


In [26]:
#Highest-Performing Schools (by % Overall Passing)
high_perform = school_df.sort_values(by="Passing math and reading score (%)", ascending=False)
high_perform.head(5)

,School name,School type,School budget,Total students,Per student budget,Average math score per school,Average reading score per school,Passing math per school (%),Passing reading per school (%),Passing math and reading score (%)
6,Cabrera High School,Charter,1081356,1858,582.0,83.061895,83.975780,94.13,97.04,91.33
14,Thomas High School,Charter,1043130,1635,638.0,83.418349,83.848930,93.27,97.31,90.95
4,Griffin High School,Charter,917500,1468,625.0,83.351499,83.816757,93.39,97.14,90.60
5,Wilson High School,Charter,1319574,2283,578.0,83.274201,83.989488,93.87,96.54,90.58
9,Pena High School,Charter,585858,962,609.0,83.839917,84.044699,94.59,95.95,90.54


In [27]:
# Lowest-Performing Schools (by % Overall Passing)
lowest_perform = school_df.sort_values(by="Passing math and reading score (%)", ascending=True)
lowest_perform.iloc[:-10 , :]

,School name,School type,School budget,Total students,Per student budget,Average math score per school,Average reading score per school,Passing math per school (%),Passing reading per school (%),Passing math and reading score (%)
11,Rodriguez High School,District,2547363,3999,637.0,76.842711,80.744686,66.37,80.22,52.99
1,Figueroa High School,District,1884411,2949,639.0,76.711767,81.158020,65.99,80.74,53.20
0,Huang High School,District,1910635,2917,655.0,76.629414,81.182722,65.68,81.32,53.51
3,Hernandez High School,District,3022020,4635,652.0,77.289752,80.934412,66.75,80.86,53.53
12,Johnson High School,District,3094650,4761,650.0,77.072464,80.966394,66.06,81.22,53.54


In [28]:
# Math Scores by Grade
m_scores_df = schools_complete.groupby(['school_name','grade'], sort=False)['math_score'].mean().reset_index(name='math_avg')
m_scores_sorted_df = m_scores_df.sort_values(["school_name"], ascending=True)

m_scores_df = m_scores_sorted_df.rename(columns={"school_name":"School name","grade":"Grade", "math_avg":"Math average score per grade"})
m_scores_df

,School name,Grade,Math average score per grade
29,Bailey High School,12th,76.492218
28,Bailey High School,9th,77.083676
30,Bailey High School,11th,77.515588
31,Bailey High School,10th,76.996772
25,Cabrera High School,9th,83.094697
24,Cabrera High School,11th,82.765560
26,Cabrera High School,12th,83.277487
27,Cabrera High School,10th,83.154506
4,Figueroa High School,10th,76.539974
6,Figueroa High School,9th,76.403037


In [29]:
# Reading Scores by Grade
read_scores_df = schools_complete.groupby(['school_name','grade'], sort=False)['reading_score'].mean().reset_index(name='read_avg')
read_scores_sorted_df = read_scores_df.sort_values(["school_name"], ascending=True)

read_scores_df = read_scores_sorted_df.rename(columns={"school_name":"School name","grade":"Grade", "read_avg":"Reading average score per grade"})
read_scores_df

,School name,Grade,Reading average score per grade
29,Bailey High School,12th,80.912451
28,Bailey High School,9th,81.303155
30,Bailey High School,11th,80.945643
31,Bailey High School,10th,80.907183
25,Cabrera High School,9th,83.676136
24,Cabrera High School,11th,83.788382
26,Cabrera High School,12th,84.287958
27,Cabrera High School,10th,84.253219
4,Figueroa High School,10th,81.408912
6,Figueroa High School,9th,81.198598


In [41]:
# Scores by School Spending
school_spending_df = school_df.drop(columns=['School type', 'School budget', 'Total students'])
school_spending_df['Cutoff groups'] = pd.qcut(school_spending_df['Per student budget'],[0, 0.25, 0.5, 0.75, 1])
school_spending_df

,School name,Per student budget,Average math score per school,Average reading score per school,Passing math per school (%),Passing reading per school (%),Passing math and reading score (%),Cutoff groups
0,Huang High School,655.0,76.629414,81.182722,65.68,81.32,53.51,"(641.5, 655.0]"
1,Figueroa High School,639.0,76.711767,81.158020,65.99,80.74,53.20,"(628.0, 641.5]"
2,Shelton High School,600.0,83.359455,83.725724,93.87,95.85,89.89,"(591.5, 628.0]"
3,Hernandez High School,652.0,77.289752,80.934412,66.75,80.86,53.53,"(641.5, 655.0]"
4,Griffin High School,625.0,83.351499,83.816757,93.39,97.14,90.60,"(591.5, 628.0]"
5,Wilson High School,578.0,83.274201,83.989488,93.87,96.54,90.58,"(577.999, 591.5]"
6,Cabrera High School,582.0,83.061895,83.975780,94.13,97.04,91.33,"(577.999, 591.5]"
7,Bailey High School,628.0,77.048432,81.033963,66.68,81.93,54.64,"(591.5, 628.0]"
8,Holden High School,581.0,83.803279,83.814988,92.51,96.25,89.23,"(577.999, 591.5]"
9,Pena High School,609.0,83.839917,84.044699,94.59,95.95,90.54,"(591.5, 628.0]"


In [55]:
# Scores by School Size
scores_by_size = school_df.loc[:, ['School name','Total students','Passing math and reading score (%)']]
scores_by_size[ 'School size'] = pd.qcut(scores_by_size['Total students'],[0, 0.33, 0.66, 1], labels=['Small', 'Medium', 'Large'])
scores_by_size

,School name,Total students,Passing math and reading score (%),School size
0,Huang High School,2917,53.51,Medium
1,Figueroa High School,2949,53.20,Large
2,Shelton High School,1761,89.89,Small
3,Hernandez High School,4635,53.53,Large
4,Griffin High School,1468,90.60,Small
5,Wilson High School,2283,90.58,Medium
6,Cabrera High School,1858,91.33,Medium
7,Bailey High School,4976,54.64,Large
8,Holden High School,427,89.23,Small
9,Pena High School,962,90.54,Small


In [57]:
# Scores by School Type
scores_by_type = school_df.loc[:, ['School name','School type','Passing math and reading score (%)']]
scores_by_type

,School name,School type,Passing math and reading score (%)
0,Huang High School,District,53.51
1,Figueroa High School,District,53.20
2,Shelton High School,Charter,89.89
3,Hernandez High School,District,53.53
4,Griffin High School,Charter,90.60
5,Wilson High School,Charter,90.58
6,Cabrera High School,Charter,91.33
7,Bailey High School,District,54.64
8,Holden High School,Charter,89.23
9,Pena High School,Charter,90.54
